# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# Load csv exported from Part I
output_file = "../output_data/cities_weather_output_final.csv"

# Read CSV into dataframe
part2_df=pd.read_csv(output_file)
part2_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Access maps with API key
gmaps.configure(api_key=g_key)

In [ ]:
# Store latitude and longitude into coordinates
coordinates = part2_df[["Lat", "Lon"]].astype(float)

# Convert humidity to to float and store
humidity = part2_df["Humidity"].astype(float)

In [ ]:
# Create humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False)
fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Ideal weather conditions
ideal_weather_df = part2_df.loc[(part2_df["Max Temp"] >= 50) &
                               (part2_df["Max Temp"] <= 70) & 
                               (part2_df["Humidity"] < 50) &
                               (part2_df["Cloudiness"] < 50)]

ideal_weather_df = ideal_weather_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Store ideal weather dataframe into new variable
hotel_df = ideal_weather_df

# Add "Hotel Name" to new dataframe
hotel_df["Hotel Name"] = ""

# Display dataframe
hotel_df

In [ ]:
# Use iterrows to iterate through new dataframe row by row
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    latitude = row["Lat"]
    longitude = row["Lon"]
    city = row["City"]
    
    address = f"{latitude}, {longitude}"
    
    # Create parameters to search for hotels within 5000 meters of city's coordinates
    params = {"location":address,
             "keyword":"hotel",
             "radius":5000,
             "type":"lodging",
             "key": g_key}
    
    # API request in json format
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except:
        
        hotel_df.loc[index, "Hotel Name"] = "N/A"
        
hotel_df

In [ ]:
# Clean dataframe to remove rows without usable data
hotel_df = hotel_df[hotel_df["Hotel Name"] != "N/A"]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig